In [1]:
#link = "https://www.youtube.com/watch?v=QRKFLqMa5gw"
#link = "https://www.youtube.com/watch?v=OpP_WNGgY1k"
link = "https://www.youtube.com/watch?v=qz2AVif8FWY"

In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from time import sleep
import os
import pandas as pd

cwd = os.getcwd()

PATH = cwd + "\ChromeDriver\chromedriver.exe"

driver = webdriver.Chrome(service=ChromeService(PATH))
driver.get(link)

prev_height = 0
scroll_counter = 100

# Scroll Down all coments

while(True):
    height = driver.execute_script('return document.documentElement.scrollHeight')
    driver.execute_script("window.scrollTo(0, " + str(height) + ");")
    
    if prev_height == height:
        scroll_counter-=1
        if scroll_counter == 0:
            break
    else:
        scroll_counter = 100
        
    prev_height = height

In [3]:
def get_content(comment):
    
    content_text = ''
    
    content = comment.find_element("xpath", ".//yt-formatted-string[@id = 'content-text']")
    #contents = comment.find_elements("xpath", "//*[@id='content-text']/span")
    
    try:
        for sub_content in content.find_element("xpath", "./span"):
            content_text = content_text + sub_content.text + " "
    except:
        content_text += content.text       
    
    return content_text

In [4]:
df = pd.DataFrame(columns = ["Comments"])

comments_XPATH = "/html/body/ytd-app/div[1]/ytd-page-manager/ytd-watch-flexy/div[5]/div[1]/div/div[2]/ytd-comments/ytd-item-section-renderer/div[3]"
comments_section = driver.find_element("xpath", comments_XPATH)
comments = comments_section.find_elements("xpath",'./ytd-comment-thread-renderer')
print(len(comments))

# Press all view replies button to comments
for comment in comments:
    try:
        see_replies = comment.find_element("xpath", './/ytd-button-renderer[@id="more-replies"]')
        see_replies.click()
    except:
        continue

        
# This is not the same as previous comments
comments = comments_section.find_elements("xpath",'.//ytd-comment-thread-renderer')

for comment in comments:
    
    content = get_content(comment)
    df.loc[len(df)] = content
    
    try:
        replies_section = comment.find_element("xpath", ".//ytd-comment-replies-renderer")
        replies = replies_section.find_elements("xpath", ".//ytd-comment-renderer")
        for reply in replies:
            content = get_content(reply)
            df.loc[len(df)] = content
    except:
        continue
        
# Fetchning the title of the video

title_path = "/html/body/ytd-app/div[1]/ytd-page-manager/ytd-watch-flexy/div[5]/div[1]/div/div[2]/ytd-watch-metadata/div/div[1]/h1/yt-formatted-string"
title = driver.find_element("xpath", title_path).text

print(df)

14
                                             Comments
0   I remember watching this game, Tottenhams play...
1   I forgot how hard Tottenham fight in this matc...
2                                   Messi is majestic
3                                       Rakitic Goal 
4   if you really think ab it…..this could’ve been...
5             Yeah unfortunately his team failed him.
6    @fabianu4019 the little ghost from argentina ...
7    @mobamba893  You are mentally tortured by the...
8                                     Much love messi
9                       Alba with 3 assists this game
10  How good is Messi...its a disgrace to football...
11                                         GooSebumps
12                                          good game
13                 MESSI EN SU SEGUNDO PATIO WEMBLEY,
14                                       I miss Messi
15                                    Spur pun trauma
16                                         MESSİ KRAL


In [6]:
file = df.to_csv("new_file.csv")

In [7]:
import boto3
s3 = boto3.resource("s3")

bucket_name = ''

for bucket in s3.buckets.all():
    bucket_name = bucket.name

In [9]:
s3_client = boto3.client("s3")
s3_client.upload_file(
    Filename = "new_file.csv",
    Bucket = bucket_name,
    Key = title + ".csv"
)